# IEEE Access Paper: A Deep Learning Approach Based on Explainable Artificial Intelligence for Skin Lesion Classification
**By: University of Engineering and Technology,Lahore,pakistan**

# Import Libraries

In [2]:
# Import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score
from torch.amp import GradScaler, autocast
import cv2
from skimage.feature import graycomatrix, graycoprops
import seaborn as sns
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries
from PIL import Image
import os
import multiprocessing as mp
from functools import partial

# GPU Information

In [3]:
!nvidia-smi

Fri Apr 25 16:01:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Step 1: Fix GPU detection
print("=== Starting GPU Setup ===")
print("Uninstalling existing PyTorch packages...")
!pip uninstall -y torch torchvision torchaudio
print("Installing PyTorch with CUDA 11.8 support...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Verify GPU availability
print("\n=== Verifying GPU Availability ===")
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("GPU not detected. Running on CPU. Please check PyTorch installation or Kaggle accelerator settings.")
    print("Falling back to CPU for now. We may switch to TensorFlow if this persists.")

# Define device early
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

=== Starting GPU Setup ===
Uninstalling existing PyTorch packages...
Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Successfully uninstalled torch-2.5.1+cu124
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
Installing PyTorch with CUDA 11.8 support...
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 74.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 95.8 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.7 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 M

# Data Collection

In [5]:
# 2. Data Collection: Load DermaMNIST dataset
print("\n=== Loading DermaMNIST Dataset ===")
dataset_path = "/kaggle/input/dermamnist/dermamnist_224.npz"
data = np.load(dataset_path)
train_images = data['train_images']
train_labels = data['train_labels'].flatten()
val_images = data['val_images']
val_labels = data['val_labels'].flatten()
test_images = data['test_images']
test_labels = data['test_labels'].flatten()

# Class names for DermaMNIST
class_names = [
    'basal cell carcinoma', 'squamous cell carcinoma', 'melanoma',
    'actinic keratosis', 'benign keratosis', 'dermatofibroma', 'vascular lesion'
]
print("Number of training images:", len(train_images))
print("Number of validation images:", len(val_images))
print("Number of test images:", len(test_images))
print("Class distribution (train):", np.bincount(train_labels))


=== Loading DermaMNIST Dataset ===
Number of training images: 7007
Number of validation images: 1003
Number of test images: 2005
Class distribution (train): [ 228  359  769   80  779 4693   99]


# Step-1: Image Preprocessing & Segmentation  
To enhance image quality and focus on the lesion areas, the following preprocessing steps are applied:​

1.  **Objective:** Enhance image quality, standardize dimensions, and focus on regions of interest (ROIs).

    I’ll preprocess images by:
    
    * Applying noise reduction (Gaussian blur).
    * Resizing to 224x224 (standard for models like ResNet).
    * Normalizing pixel values to [0, 1].
    * Cropping ROIs (simplified to central cropping for now).
1. **Implementation:**

Since the dataset is large, we’ll preprocess images on-the-fly during training to save memory, using a generator.
We’ll assume metadata.csv has columns like isic_id (image filename without extension) and diagnosis (label).

In [6]:
# 3. Preprocessing: Precompute Segmentation and GLCM Features with Multiprocessing
print("\n=== Preprocessing: Segmentation and GLCM Feature Extraction ===")

def segment_image(image):
    img = np.array(image)
    mask = np.zeros(img.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    height, width = img.shape[:2]
    rect = (10, 10, width-20, height-20)
    # Reduce iterations for faster processing (from 5 to 3)
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 3, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask2 = cv2.dilate(mask2, kernel, iterations=1)
    segmented = img * mask2[:, :, np.newaxis]
    return segmented

def compute_glcm_features(image):
    img = np.array(Image.fromarray(image).convert('L'))
    glcm = graycomatrix(img, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    features = {
        'contrast': graycoprops(glcm, 'contrast')[0, 0],
        'correlation': graycoprops(glcm, 'correlation')[0, 0],
        'energy': graycoprops(glcm, 'energy')[0, 0],
        'homogeneity': graycoprops(glcm, 'homogeneity')[0, 0]
    }
    return np.array([features['contrast'], features['correlation'], features['energy'], features['homogeneity']])

def process_image(img, idx, total):
    if idx % 500 == 0:
        print(f"Processed {idx}/{total} images...")
    segmented = segment_image(img)
    glcm = compute_glcm_features(segmented)
    return segmented, glcm

# Precompute and save segmented images and GLCM features using multiprocessing
output_dir = "/kaggle/working/preprocessed_dermamnist"
os.makedirs(output_dir, exist_ok=True)

def preprocess_and_save(images, split_name):
    segmented_path = os.path.join(output_dir, f"{split_name}_segmented.npy")
    glcm_path = os.path.join(output_dir, f"{split_name}_glcm.npy")
    
    if os.path.exists(segmented_path) and os.path.exists(glcm_path):
        print(f"Loading precomputed {split_name} data...")
        segmented_images = np.load(segmented_path)
        glcm_features = np.load(glcm_path)
    else:
        print(f"Preprocessing {split_name} data ({len(images)} images) with multiprocessing...")
        num_cores = mp.cpu_count()
        print(f"Using {num_cores} CPU cores for preprocessing.")
        pool = mp.Pool(processes=num_cores)
        process_func = partial(process_image, total=len(images))
        results = pool.starmap(process_func, [(img, i) for i, img in enumerate(images)])
        pool.close()
        pool.join()
        segmented_images = np.array([r[0] for r in results])
        glcm_features = np.array([r[1] for r in results])
        # Save to disk
        np.save(segmented_path, segmented_images)
        np.save(glcm_path, glcm_features)
        print(f"Saved precomputed {split_name} data.")
    
    return segmented_images, glcm_features

# Preprocess all splits
print("Preprocessing training set...")
train_segmented, train_glcm = preprocess_and_save(train_images, "train")
print("Preprocessing validation set...")
val_segmented, val_glcm = preprocess_and_save(val_images, "val")
print("Preprocessing test set...")
test_segmented, test_glcm = preprocess_and_save(test_images, "test")

# Custom Dataset for DermaMNIST
class DermaMNISTDataset(Dataset):
    def __init__(self, images, glcm_features, labels, transform=None):
        self.images = images
        self.glcm_features = glcm_features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])
        glcm_features = self.glcm_features[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, glcm_features, label


=== Preprocessing: Segmentation and GLCM Feature Extraction ===
Preprocessing training set...
Preprocessing train data (7007 images) with multiprocessing...
Using 4 CPU cores for preprocessing.
Processed 0/7007 images...
Processed 500/7007 images...
Processed 1000/7007 images...
Processed 1500/7007 images...
Processed 2000/7007 images...
Processed 2500/7007 images...
Processed 3000/7007 images...
Processed 3500/7007 images...
Processed 4000/7007 images...
Processed 4500/7007 images...
Processed 5000/7007 images...
Processed 5500/7007 images...
Processed 6000/7007 images...
Processed 6500/7007 images...
Processed 7000/7007 images...
Saved precomputed train data.
Preprocessing validation set...
Preprocessing val data (1003 images) with multiprocessing...
Using 4 CPU cores for preprocessing.
Processed 0/1003 images...
Processed 500/1003 images...
Processed 1000/1003 images...
Saved precomputed val data.
Preprocessing test set...
Preprocessing test data (2005 images) with multiprocessing.

# Step-2: Data Augmentation
To increase data diversity and prevent overfitting, the following augmentation techniques are applied:​

* **Rotation:** Random rotations at various angles.​

* **Flipping:** Horizontal and vertical flips.​

* **Cropping:** Random crops to simulate zoom.​

* **Brightness and Contrast Adjustment:** Randomly altering brightness and contrast levels.​

* **Noise Addition:** Introducing random noise to images.

In [16]:
# Transforms
train_transforms = transforms.Compose([
    transforms.RandomAffine(degrees=30, shear=10, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.1))
])
val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Step-3: Model Training with ResNet-18
Utilizing transfer learning, a pre-trained ResNet-18 model is fine-tuned for skin lesion classification. The final fully connected layer is modified to output predictions for the nine classes present in the ISIC 2019 dataset.

In [17]:
# Create datasets
print("\n=== Creating Datasets and DataLoaders ===")
train_dataset = DermaMNISTDataset(train_segmented, train_glcm, train_labels, transform=train_transforms)
val_dataset = DermaMNISTDataset(val_segmented, val_glcm, val_labels, transform=val_transforms)
test_dataset = DermaMNISTDataset(test_segmented, test_glcm, test_labels, transform=val_transforms)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True)
print(f"Created DataLoaders with batch size {batch_size}")

# 4. Classification: ResNet18 + MLP for Feature Fusion
print("\n=== Setting Up Model ===")

class CombinedModel(nn.Module):
    def __init__(self, num_classes, glcm_feature_dim=4):
        super(CombinedModel, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        resnet_fc_dim = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        self.glcm_fc = nn.Linear(glcm_feature_dim, 64)
        self.combined_fc = nn.Sequential(
            nn.Linear(resnet_fc_dim + 64, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x, glcm_features):
        resnet_features = self.resnet(x)
        glcm_features = self.glcm_fc(glcm_features)
        combined = torch.cat((resnet_features, glcm_features), dim=1)
        output = self.combined_fc(combined)
        return output

# Normalize GLCM features
print("Normalizing GLCM features...")
scaler = StandardScaler()
scaler.fit(train_glcm)

# Compute class weights
print("Computing class weights...")
class_counts = np.bincount(train_labels)
class_weights = torch.tensor([1.0 / count for count in class_counts], dtype=torch.float)
class_weights = class_weights / class_weights.sum() * len(class_counts)
class_weights = class_weights.to(device)

# Initialize model
model = CombinedModel(num_classes=len(class_names)).to(device)
print(f"Model initialized on device: {next(model.parameters()).device}")

# Loss, optimizer, and scaler
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scaler = GradScaler('cuda' if torch.cuda.is_available() else 'cpu')

# Training loop
print("\n=== Starting Training ===")
num_epochs = 15
best_val_acc = 0.0
for epoch in range(num_epochs):
    print(f"\nStarting Epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader):
        if i % 50 == 0:
            print(f"  Batch {i}/{len(train_loader)}")
        inputs, glcm_features, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        glcm_features = scaler.transform(glcm_features.numpy())
        glcm_features = torch.tensor(glcm_features, dtype=torch.float32).to(device)
        optimizer.zero_grad()
        with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            outputs = model(inputs, glcm_features)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

    # Validation
    print(f"Validating Epoch {epoch+1}...")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, glcm_features, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            glcm_features = scaler.transform(glcm_features.numpy())
            glcm_features = torch.tensor(glcm_features, dtype=torch.float32).to(device)
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(inputs, glcm_features)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Validation Acc: {val_acc:.2f}%")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("Saved best model with validation accuracy:", best_val_acc)
    if val_acc > 95:
        print("Validation accuracy > 95%, stopping training.")
        break

# Clear memory after training
torch.cuda.empty_cache()
print("Training complete, GPU memory cleared.")

NameError: name 'device' is not defined

In [14]:
# 4. Classification: ResNet18 + MLP for Feature Fusion
class CombinedModel(nn.Module):
    def __init__(self, num_classes, glcm_feature_dim=4):
        super(CombinedModel, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        resnet_fc_dim = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        self.glcm_fc = nn.Linear(glcm_feature_dim, 64)
        self.combined_fc = nn.Sequential(
            nn.Linear(resnet_fc_dim + 64, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x, glcm_features):
        resnet_features = self.resnet(x)
        glcm_features = self.glcm_fc(glcm_features)
        combined = torch.cat((resnet_features, glcm_features), dim=1)
        output = self.combined_fc(combined)
        return output

In [15]:
# Normalize GLCM features
scaler = StandardScaler()
glcm_features_list = []
for idx in range(len(train_dataset)):
    _, glcm_features, _ = train_dataset[idx]
    glcm_features_list.append(glcm_features)
glcm_features_list = np.array(glcm_features_list)
scaler.fit(glcm_features_list)

# Compute class weights
class_counts = np.bincount(train_labels)
class_weights = torch.tensor([1.0 / count for count in class_counts], dtype=torch.float)
class_weights = class_weights / class_weights.sum() * len(class_counts)
class_weights = class_weights.to(device)

AttributeError: 'Image' object has no attribute 'shape'

In [ ]:
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CombinedModel(num_classes=len(class_names)).to(device)
print(f"Training on device: {next(model.parameters()).device}")

# Loss, optimizer, and scaler
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scaler = GradScaler('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Training loop
num_epochs = 15
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for data in train_loader:
        inputs, glcm_features, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        glcm_features = scaler.transform(glcm_features.numpy())
        glcm_features = torch.tensor(glcm_features, dtype=torch.float32).to(device)
        optimizer.zero_grad()
        with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            outputs = model(inputs, glcm_features)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

In [ ]:
# Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, glcm_features, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            glcm_features = scaler.transform(glcm_features.numpy())
            glcm_features = torch.tensor(glcm_features, dtype=torch.float32).to(device)
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(inputs, glcm_features)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Validation Acc: {val_acc:.2f}%")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
    if val_acc > 95:
        break

# Clear memory after training
torch.cuda.empty_cache()

# Model Explainability with the LIME + GradCam
To interpret the model's predictions, LIME (Local Interpretable Model-Agnostic Explanations) is employed. LIME provides visual explanations by highlighting regions in the image that most influenced the model's decision.

In [ ]:
# Grad-CAM implementation
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.hook_handles = []
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0]
        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))

    def __call__(self, x, glcm_features, index=None):
        self.model.eval()
        output = self.model(x, glcm_features)
        if index is None:
            index = torch.argmax(output, dim=1)
        self.model.zero_grad()
        score = output[0, index]
        score.backward()
        weights = torch.mean(self.gradients, dim=[2, 3], keepdim=True)
        cam = torch.sum(weights * self.activations, dim=1)
        cam = torch.relu(cam)
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)
        return cam.cpu().detach().numpy()

# Explainability
model.eval()
sample_idx = 0
sample_image = Image.fromarray(val_images[sample_idx])
sample_image = segment_image(sample_image)
sample_glcm = compute_glcm_features(sample_image)
sample_glcm = scaler.transform([sample_glcm])[0]
sample_glcm = torch.tensor(sample_glcm, dtype=torch.float32).unsqueeze(0).to(device)

# Preprocess for LIME and Grad-CAM
lime_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
sample_tensor = lime_transform(sample_image).unsqueeze(0).to(device)

# LIME
def predict_fn(images):
    images = torch.from_numpy(images.transpose((0, 3, 1, 2))).float().to(device)
    images = images / 255.0
    images = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(images)
    glcm_batch = sample_glcm.repeat(images.shape[0], 1)
    with torch.no_grad():
        outputs = model(images, glcm_batch)
    return outputs.cpu().numpy()

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(
    np.array(sample_image),
    predict_fn,
    top_labels=5,
    num_samples=500
)
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5)
plt.imshow(mark_boundaries(np.array(sample_image)/255.0, mask))
plt.title("LIME Explanation")
plt.show()

# Grad-CAM
gradcam = GradCAM(model, model.resnet.layer4[-1])
cam = gradcam(sample_tensor, sample_glcm)
cam = cv2.resize(cam[0], (224, 224))
plt.imshow(np.array(sample_image))
plt.imshow(cam, cmap='jet', alpha=0.5)
plt.title("Grad-CAM")
plt.show()

torch.cuda.empty_cache()

# Evaluation Matrix

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate on test set
model.eval()
all_preds = []
all_labels = []
all_probs = []
with torch.no_grad():
    for data in test_loader:
        inputs, glcm_features, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        glcm_features = scaler.transform(glcm_features.numpy())
        glcm_features = torch.tensor(glcm_features, dtype=torch.float32).to(device)
        with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            outputs = model(inputs, glcm_features)
        probs = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# Compute metrics
accuracy = 100 * (np.array(all_preds) == np.array(all_labels)).mean()
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# ROC-AUC
roc_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')
print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

torch.cuda.empty_cache()

# Visualizations

In [ ]:
from matplotlib.patches import FancyArrowPatch

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0, 10)
ax.set_ylim(0, 6)
ax.axis('off')

stages = [
    ("Data\nCollection", 1, 5),
    ("Segmentation", 3, 5),
    ("Feature\nExtraction", 5, 5),
    ("Classification", 7, 5),
    ("Evaluation", 7, 3),
    ("XAI", 5, 3)
]

for text, x, y in stages:
    ax.text(x, y, text, ha='center', va='center', bbox=dict(facecolor='lightblue', edgecolor='black'))

for i in range(len(stages)-1):
    x1, y1 = stages[i][1], stages[i][2]
    x2, y2 = stages[i+1][1], stages[i+1][2]
    if y1 == y2:
        arrow = FancyArrowPatch((x1+0.5, y1), (x2-0.5, y2), arrowstyle='->', mutation_scale=20, color='black')
    else:
        arrow = FancyArrowPatch((x1, y1-0.5), (x2, y2+0.5), arrowstyle='->', mutation_scale=20, color='black')
    ax.add_patch(arrow)

plt.title("Pipeline Flowchart for DermaMNIST")
plt.show()